In [34]:
!pip install arxiv


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


# Langchain TOOL

In [35]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [36]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [37]:
wiki.name

'wikipedia'

# BUild Custom TOOL

In [38]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

#loader=WebBaseLoader("https://docs.smith.langchain.com/")
loader=WebBaseLoader("https://docs.smith.langchain.com/pricing")

docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x13dc3beb0>)

In [39]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [55]:
retriever_tool.name

'langsmith_search'

# Langchain TOOL

In [41]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [53]:
tools=[wiki,arxiv,retriever_tool]

In [54]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/trishulchowdhury/Documents/GitHub/ALL-Langchain-main/venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10aa1fbe0>, retriever=VectorStoreRetriever(t

In [44]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

## Agents

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

#### This is basically the hub of Prompts 

###### You can change the Prompt

In [56]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [46]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [47]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [62]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Pricing | 🦜️🛠️ LangSmith

available on Discord.On the Plus plan, you will also receive preferential, email support at support@langchain.dev for LangSmith-related questions only and we'll do our best to respond within the next business day.On the Enterprise plan, you’ll get white-glove support with a Slack channel, a dedicated customer success manager, and monthly check-ins to go over LangSmith and LangChain questions. We can help with anything from debugging, agent and RAG techniques, evaluation approaches, and cognitive architecture reviews. If you purchase the add-on to run LangSmith in your environment, we’ll also support deployments and new releases with our infra engineering team on-call.Where is my data stored?​You may choose to sign up in either the US or EU region. See the cloud architecture reference for more details. If you’re on the Enterprise plan, we can deliver LangSmith 

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a tool available on Discord that offers different plans for users, including Plus and Enterprise plans. The Plus plan provides preferential email support, while the Enterprise plan offers white-glove support with a dedicated customer success manager and monthly check-ins. LangSmith is compliant with security frameworks such as SOC 2 Type II, GDPR, and HIPAA.\n\nUsers can choose to store their data in either the US or EU region. For Enterprise plan users, LangSmith can be delivered to run on their Kubernetes cluster in AWS, GCP, or Azure to ensure data remains within their environment. LangSmith does not train on user data, and users retain all rights to their data.\n\nFor pricing, LangSmith offers different billing options, including monthly self-serve credit card billing and annual invoicing with custom terms and data privacy agreements. Users can purchase LangSmith credits for tracing usage, and personal accounts without a

In [63]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges come from a finite set of allocations.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges come from a finite set of allocations.'}

In [64]:
agent_executor.invoke({"input":"Tell me about Langsmith price details"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith price details'}`


Pricing | 🦜️🛠️ LangSmith

details. If you’re on the Enterprise plan, we can deliver LangSmith to run on your kubernetes cluster in AWS, GCP, or Azure so that data never leaves your environment.Which security frameworks is LangSmith compliant with?​We are SOC 2 Type II, GDPR, and HIPAA compliant.You can request more information about our security policies and posture at trust.langchain.com. Please note we only enter into BAAs with customers on our Enterprise plan.Will you train on the data that I send LangSmith?​We will not train on your data, and you own all rights to your data. See LangSmith Terms of Service for more information.Was this page helpful?You can leave detailed feedback on GitHub.PreviousLangChain off-the-shelf evaluatorsNextSelf-hostingPlansPlan ComparisonQuestions and AnswersI’ve been using LangSmith since before pricing took effect for new users. When will

{'input': 'Tell me about Langsmith price details',
 'output': "LangSmith offers different pricing plans based on the user's needs. Here are the details of the pricing plans:\n\n1. **Startups Plan**:\n   - Designed for early stage startups building AI applications.\n   - Free for 1 user with 5,000 free traces per month.\n   - Additional traces are billed starting at 0.05¢ per trace.\n\n2. **Developer Plan**:\n   - Designed for hobbyists who want to start their adventure solo.\n   - Includes everything in the Developer plan, plus team features and higher rate limits.\n   - Priced at $39 per user with 10,000 free traces per month.\n   - Additional traces are billed starting at 0.05¢ per trace.\n\n3. **Enterprise Plan**:\n   - Designed for teams with more security, deployment, and support needs.\n   - Custom pricing based on specific requirements.\n   - Offers features like debugging traces, dataset collection, testing and evaluation, architectural guidance, dedicated customer success mana